In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from threading import Thread
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))
#        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train_dir = "/home/oscar/Escritorio/predict-volcanic-eruptions-ingv-oe/train" # Oscar
train_filenames = os.listdir(train_dir)

In [6]:
train_filename = train_filenames[0]
data = pd.read_csv( os.path.join(train_dir,train_filename), dtype="Int16")
data.describe()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
count,60001.000000,60001.000000,0,60001.000000,60001.000000,60001.000000,60001.00000,60001.000000,0,60001.000000
mean,7.593423,2.545708,<NA>,0.661839,-0.458976,1.431776,1.21018,3.117231,<NA>,-7.668739
std,387.653854,553.938047,<NA>,319.048016,170.261498,1016.208891,396.16904,594.306992,<NA>,669.995033
min,-3448.000000,-7088.000000,<NA>,-3628.000000,-1452.000000,-3690.000000,-2132.00000,-2444.000000,<NA>,-4662.000000
25%,-210.000000,-229.000000,<NA>,-186.000000,-92.000000,-678.000000,-236.00000,-400.000000,<NA>,-358.000000
50%,0.000000,0.000000,<NA>,0.000000,0.000000,0.000000,0.00000,0.000000,<NA>,0.000000
75%,213.000000,232.000000,<NA>,181.000000,93.000000,684.000000,243.00000,413.000000,<NA>,356.000000
max,3478.000000,9804.000000,<NA>,3141.000000,1457.000000,4240.000000,1843.00000,2243.000000,<NA>,3951.000000


# Checando que tan buena estuvo la prank

Según distintos archivos, hay ciertos que no tienen valores registrados (NaNs) o cosas raras como puros ceros. Sería importante ver como está dicha situación para todos los datos

In [2]:
def check_prank(data):
    """Function to check if some columns contain only nans or some constant"""
    nrows, _ = data.shape
    only_nans = [data[sensor].isna().sum() for sensor in data.columns]
    only_constants = [data[sensor].std() == 0 for sensor in data.columns]
    
    return only_nans, only_constants

In [3]:
def yield_partition(data, npartitions):
    n_elements = len(data)
    n = n_elements//npartitions
    
    for k in range(npartitions):
        i = k*n
        j = (k+1)*n
        if k != npartitions - 1:
            yield data[i:j]
        else:
            yield data[i:]  # Last one

In [4]:
def parallelize(nthreads):
    def decorator(original_function):
        def parallelized_function(dir_path, filenames, verbose=True):
            parallel_ans = []
            threads = []
            counter = 1
    
            # Partir los datos
            for partition in yield_partition(filenames, nthreads):
                thread = Thread(target=original_function, args=(dir_path, partition, verbose, parallel_ans))
                threads.append(thread)
                print("Starting",counter)
                thread.start()
                counter += 1 
            
            for thread in threads:
                thread.join()
            
            list_nans = [df_nan for df_nan, df_constant in parallel_ans]
            list_constants = [df_constant for df_nan, df_constant in parallel_ans]
            concatenation_nans = pd.concat(list_nans)
            concatenation_constants = pd.concat(list_constants)
            return concatenation_nans, concatenation_constants
        
        return parallelized_function
    
    return decorator

In [5]:
@parallelize(8)
def check_pranks(dir_path, filenames, verbose=True, parallel_ans = None):
    nans = []
    constants = []
    for i,filename in enumerate(filenames,1):
        df = pd.read_csv(os.path.join(dir_path,filename), dtype="Int16")
        only_nans, only_constants = check_prank(df)
        nans.append(only_nans)
        constants.append(only_constants)
        if verbose and i%10 == 0:
            print(i)
    
    df_nans = pd.DataFrame(nans, columns=df.columns)
    df_constants = pd.DataFrame(constants, columns=df.columns)
    
    if parallel_ans != None:
        parallel_ans.append( (df_nans, df_constants) )
    else:
        return df_nans, df_constants

## Datos de entrenamiento 

In [ ]:
%%time
df_train_nans, df_train_constants = check_pranks(train_dir, train_filenames)

In [ ]:
df_train_nans.to_csv("df_train_nans.csv", index=False)

In [ ]:
df_train_nans.sum(axis=0)

In [ ]:
df_train_constants.sum(axis=0)

In [ ]:
df_train_constants.to_csv("df_train_constants.csv", index=False)

## Datos de prueba

In [6]:
test_dir = "/home/oscar/Escritorio/predict-volcanic-eruptions-ingv-oe/test"
test_filenames = os.listdir(test_dir)

In [7]:
%%time
df_test_nans, df_test_constants = check_pranks(test_dir, test_filenames)

Starting 1
Starting 2
Starting 3
Starting 4
Starting 5
Starting 6
Starting 7
Starting 8
10
10
10
10
10
10
10
10
20
20
20
2020

20
20
20
30
30
30
30
30
30
30
30
40
40
40
40
40
40
40
40
50
50
50
50
50
50
50
50
60
60
60
60
60
60
60
60
70
70
70
70
70
70
70
70
80
80
80
80
80
80
80
80
90
90
90
9090

90
90
90
100
100
100
100100

100
100
100
110
110
110
110
110
110
110
110
120
120
120
120
120
120120

120
130
130
130
130
130
130
130
130
140
140
140
140
140140

140
140
150
150
150
150
150
150
150
150
160160

160
160
160
160
160
160
170
170170

170170
170

170
170
180
180
180
180
180
180
180180

190
190
190
190
190
190
190
190
200200

200
200200

200
200200

210
210
210
210
210210

210
210
220
220
220
220
220220

220
220
230
230
230
230
230
230230

230
240
240
240
240
240240

240
240
250250250


250
250
250
250
250
260
260
260
260260

260260

260
270
270
270
270
270
270
270
270
280
280
280
280
280
280
280280

290
290
290
290
290
290
290
290
300
300
300
300
300
300
300
300
310
310
310
310
310
310


In [10]:
df_test_nans.to_csv("df_test_nans.csv", index=False)
df_test_constants.to_csv("df_test_constants.csv", index=False)

In [8]:
df_test_nans.sum(axis=0)

sensor_1     23328531
sensor_2     75807628
sensor_3     26593834
sensor_4        13406
sensor_5     92651510
sensor_6        57658
sensor_7      2713162
sensor_8     17653700
sensor_9     51208744
sensor_10    46050548
dtype: int64

In [9]:
df_test_constants.sum(axis=0)

sensor_1       0
sensor_2       0
sensor_3       0
sensor_4       0
sensor_5       0
sensor_6       0
sensor_7       0
sensor_8       0
sensor_9       0
sensor_10    815
dtype: int64

----------------
# Zona después del \end{document}

`time_to_eruption` es el número de muestras antes de la siguiente erupción

## Estandarizando los datos

In [1]:
from os import path, listdir
import pandas as pd
import numpy as np
path_originales = "/home/oscar/Escritorio/predict-volcanic-eruptions-ingv-oe"
path_copias = "/home/oscar/Escritorio/estandarizados" 

In [39]:
def estandarizar(df):
    aggs = df.agg([np.nanmean, np.nanstd]).astype("Float16")
    standarized_df = (df - aggs.loc["nanmean",:])/ aggs.loc["nanstd",:]
    return standarized_df

In [36]:
train = path.join(path_originales,"train")
file = listdir(train)[1]
arch = pd.read_csv(path.join(train, file))
#arch.describe()
s_df = estandarizar(arch)
s_df.astype("float16")
#s_df.to_csv(path.join(path.join(path_copias, "train"), file), index = False)

           sensor_1    sensor_2    sensor_3    sensor_4    sensor_5  \
nanmean   -1.207031   -0.921875    1.230469    1.071289    0.858887   
nanstd   401.000000  444.000000  452.250000  388.500000  206.250000   

           sensor_6    sensor_7   sensor_8    sensor_9   sensor_10  
nanmean   11.429688    6.132812   -0.67334    0.802246   -8.828125  
nanstd   760.000000  870.000000  810.50000  337.750000  783.500000  


,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
0,2.127698,0.911986,0.211762,-2.218974,1.600684,-0.126881,0.455020,0.047715,-0.778097,0.974892
1,1.933185,0.972797,0.320110,-2.211252,1.736442,0.127066,0.181457,0.283372,-0.502745,0.858747
2,1.796028,0.821896,0.227241,-2.103144,1.498866,0.256014,-0.199003,0.332725,-0.162257,0.761746
3,1.678821,0.504329,0.110049,-2.077404,0.999472,0.286277,-0.435785,0.399350,-0.123767,0.639219
4,1.469344,0.080905,0.160906,-2.105718,0.432199,0.382329,-0.292107,0.289541,-0.366550,0.470744
...,...,...,...,...,...,...,...,...,...,...
59996,-0.281279,0.004329,-0.433898,1.225042,-0.299922,-0.462407,-1.181762,-0.112679,0.270016,-0.340998
59997,-0.231404,-0.189365,-0.139813,1.338298,-0.207801,-0.257144,-0.763371,-0.071964,0.329231,-0.222300
59998,-0.159085,-0.297473,0.355488,1.423240,-0.154467,-0.349250,-0.476015,-0.071964,0.181192,-0.131681
59999,-0.034396,-0.365041,0.534593,1.418092,-0.159316,-0.459776,-0.199003,-0.048521,0.000586,-0.016812


In [40]:
%%time

for carpeta in ["train", "test"]:
    camino = path.join(path_originales, carpeta)
    print(f"Comienzo con {carpeta}")
    for file in listdir(camino):
        lectura = path.join(camino, file)
        escritura = path.join(path.join(path_copias, carpeta), file)
        df = pd.read_csv(lectura)
        s_df = estandarizar(df).astype("Float16")
        s_df.to_csv(escritura, index = False)

Comienzo con train
Comienzo con test
CPU times: user 56min 38s, sys: 32.9 s, total: 57min 11s
Wall time: 57min 29s
